In [207]:
from gurobipy import *
import numpy as np

'''
A gardener wants to maximize the height of the stem of a tulip she plants.
She needs to decide (t) between buying two different types of tulips:
    1. Red Tulip (Tulipa orphanidea, t = 1): Average height is 15cm, std dev is 5cm
    2. Purple Tulip (Tulipa saxatilis, t = 0): Average height is 15cm, std dev is 10cm
Additionally, she will decide how many mL of water/week (x) to give the tulip (250mL-1L)
    1. For the red tulip, more water increases the average height by a factor of 0.0012x
       and increases the std dev by a factor of 0.01x
    2. For the purple tulip, more water increases the height by a factor of 0.001x and
       and decreases the std dev by a factor of 0.005x
'''
# Create model 
model1 = Model("Model #1")
model1.setParam('OutputFlag', 0)

# Create Variables 
t = model1.addVar(vtype = GRB.BINARY, name = "Tulip Type")
x = model1.addVar(lb = 250, ub = 1000, name = "Amount of Water/week (mL)")
avg = model1.addVar(name = "Average")
stdev = model1.addVar(name = "Standard Deviation")

# Monte Carlo-ing Expectation
n = 1000
i = 0
navg = 0 # For testing
obj = LinExpr()
while i < n:
    norm = np.random.standard_normal()
    navg += norm
    obj += avg + stdev*norm
    i += 1
print("Standard normal sample average: " + str(navg/n))

# Set objective
model1.setObjective((1/n)*obj, GRB.MAXIMIZE)

# Set constraints 
M = 1000

# If t = 1, avg = 15 + 0.001x, else avg = 15 + 0.0015x
model1.addConstr(avg <= 15 + 0.0012*x + M*(1 - t))
model1.addConstr(avg >= 15 + 0.0012*x - M*(1 - t))
model1.addConstr(avg <= 15 + 0.001*x + M*t)
model1.addConstr(avg >= 15 + 0.001*x - M*t)

# If t = 1, stdev = 5 + 0.01x, else avg = 10 - 0.01x
model1.addConstr(stdev <= 5 + 0.01*x + M*(1 - t))
model1.addConstr(stdev >= 5 + 0.01*x - M*(1 - t))
model1.addConstr(stdev <= 10 - 0.005*x + M*t)
model1.addConstr(stdev >= 10 - 0.005*x - M*t)

# Optimize model
model1.optimize()

# Print results 
for v in model1.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Stem Height: %g' % model1.objVal)


Standard normal sample average: -0.04829957089852673
Tulip Type: 0
Amount of Water/week (mL): 1000
Average: 16
Standard Deviation: 5
Stem Height: 15.7585


In [209]:
from gurobipy import *
import numpy as np

'''
The gardener now decides she wants to maximize the sum of the heights of the stem and flower.
She adds whether to plant the tulip outside (d = 1) in a flower bed to her list of considerations.
* Note: planting the tulip outside always adds 1 to the std dev of any distribution 
  since the outdoors present more variability 
      (i.e flower can be eaten by an animal, or thrive because of rich soil etc.)
Additionally, she considers the positive proportional relationship between total leaf surface area (lsa) 
and stem height:
    Stem Height ~ N(0.1*lsa, 0.01*lsa)
Each tulip type's leaf surface area and flower height are normally distributed and are affected by amount of water
and whether or not the plant is outside or not. 
   lsa ~ N(100 + 0.04*x + 5*d, 100 + 0.01*x + d) (if t = 1, total leaf SA ranges from 80-280cm^2)
   lsa ~ N(145 + 0.01*x + 5*d, 33 - 0.005*x + d) (if t = 0, total leaf SA ranges from 120-185cm^2)
       - More water increases lsa because there is more water available for photosynthesis 
         (photosynthesis occurs in the leaves)
       - Being planted outdoors increases lsa because there is more competition which causes 
         the plant to direct more growth to the leaf to be able to get more sunlight
    Flower Height ~ N(3 - 0.001*x + 1.5*(1 - d), 1.35 + d) (if t = 1, petal height is 2-4.7cm)
                  ~ N(3 - 0.0015*x + 0.5*(1 - d), 0.75 + d) (if t = 0, petal height is 2-3.5cm)
        - In drought conditions, plants direct more growth to their reproductive organ (flower)
          than leaf growth because larger lsa increases water loss and to ensure the species remains
          in the area (via reproduction) should it die during a drought. Simulate drought conditions 
          (less water) to induce flower growth
        - Being indoors increases flower height because there is less competition so the plant doesn't
          have to direct most growth to leaf surface area 

'''
# Create model 
model2 = Model("Model #2")
model2.setParam('OutputFlag', 0)

# Create Variables 
t = model2.addVar(vtype = GRB.BINARY, name = "Tulip Type")
x = model2.addVar(lb = 250, ub = 1000, name = "Amount of Water/week (mL)")
d = model2.addVar(vtype = GRB.BINARY, name = "Outdoor?")
lsaavg = model2.addVar(name = "Total Leaf Surface Area Average")
lsastdev = model2.addVar(name = "Total Leaf Surface Area Standard Deviation")
lsa = model2.addVar(name = "Total Leaf Surface Area")
flavg = model2.addVar(name = "Flower Petal Height Average")
flstdev = model2.addVar(name = "Flower Petal Height Standard Deviation")

# Monte Carlo-ing Expectation
n = 100
i = 0
# Store sample averages for testing 
lavg = 0
savg = 0
favg = 0
leaf = LinExpr()
stem = LinExpr()
flower = LinExpr()
while i < n:
    lnorm = np.random.standard_normal()
    snorm = np.random.standard_normal()
    fnorm = np.random.standard_normal()
    
    leaf += lsaavg + lsastdev*lnorm
    stem += 0.1*(lsa) + 0.01*(lsa)*snorm
    flower += flavg + flstdev*fnorm 
    
    lavg += lnorm
    savg += snorm 
    favg += fnorm
    
    i += 1

lavg = (1/n)*lavg
savg = (1/n)*savg
favg = (1/n)*favg

print("Leaf standard normal sample average: " + str(lavg))
print("Stem standard normal sample average: " + str(savg))
print("Flower standard normal sample average: " + str(favg) + "\n")

# Set objective
model2.setObjective((1/n)*(stem + flower), GRB.MAXIMIZE)

# Set constraints 
M = 10000

# Leaf Constraints 
# If t = 1, lsaavg = 100 + 0.06*x + 5*d, else lsaavg = 145 + 0.01*x + 5*d
model2.addConstr(lsaavg <= 100 + 0.04*x + 5*d + M*(1 - t))
model2.addConstr(lsaavg >= 100 + 0.04*x + 5*d - M*(1 - t))
model2.addConstr(lsaavg <= 145 + 0.01*x + 5*d + M*t)
model2.addConstr(lsaavg >= 145 + 0.01*x + 5*d - M*t)
# If t = 1, lsastdev = 100 + 0.01*x + d, else lsastdev = 33 - 0.005*x + d
model2.addConstr(lsastdev <=  100 + 0.01*x + d + M*(1 - t))
model2.addConstr(lsastdev >=  100 + 0.01*x + d - M*(1 - t))
model2.addConstr(lsastdev <= 33 - 0.005*x + d + M*t)
model2.addConstr(lsastdev >= 33 - 0.005*x + d - M*t)
# Setting lsa = leaf expression 
model2.addConstr(lsa <= (1/n)*leaf)
model2.addConstr(lsa >= (1/n)*leaf)

# Flower Constraints 
# If t = 1, flavg = 3 - 0.001*x + 1.5*(1 - d), else flavg = 3 - 0.0015*x + 0.5*(1 - d)
model2.addConstr(flavg <= 3 - 0.001*x + 1.5*(1 - d) + M*(1 - t))
model2.addConstr(flavg >= 3 - 0.001*x + 1.5*(1 - d) - M*(1 - t))
model2.addConstr(flavg <= 3 - 0.0015*x + 0.5*(1 - d) + M*t)
model2.addConstr(flavg >= 3 - 0.0015*x + 0.5*(1 - d) - M*t)
# If t = 1, flstdev = 1.35 + d, else flstdev = 0.75 + d
model2.addConstr(flstdev <= 1.35 + d + M*(1 - t))
model2.addConstr(flstdev >= 1.35 + d - M*(1 - t))
model2.addConstr(flstdev <= 0.75 + d + M*t)
model2.addConstr(flstdev >= 0.75 + d - M*t)

# Optimize model
model2.optimize()

# Print and check results
checkLA = 100*t.x + 0.04*x.x*t.x + 5*d.x*t.x + 145*(1-t.x) + 0.01*x.x*(1-t.x) + 5*d.x*(1-t.x)
checkLSD = 100*t.x + 0.01*x.x*t.x + d.x*t.x + 33*(1-t.x) - 0.005*x.x*(1-t.x) + d.x*(1-t.x)
checkFA = 3*t.x - 0.001*x.x*t.x + 1.5*(1-d.x)*t.x + 3*(1-t.x) - 0.0015*x.x*(1-t.x) + 0.5*(1-d.x)*(1 - t.x)
checkFSD = 1.35*t.x + d.x*t.x + 0.75*(1-t.x) + d.x*(1-t.x)
checkLeafSA = checkLA + checkLSD*lavg
for v in model2.getVars():
    print('Gurobi Result: %s: %g' % (v.varName, v.x))
    if v.varName == "Total Leaf Surface Area Average":
        print('\tCheck Result: ' + str(checkLA))
    elif v.varName == 'Total Leaf Surface Area Standard Deviation':
        print('\tCheck Result: ' + str(checkLSD))
    elif v.varName == 'Total Leaf Surface Area':
        print('\tCheck Result: ' + str(checkLeafSA))
    elif v.varName == 'Flower Petal Height Average':
        print('\tCheck Result: ' + str(checkFA))
    elif v.varName == 'Flower Petal Height Standard Deviation':
        print('\tCheck Result: ' + str(checkFSD))


print('\nGurobi Stem + Flower Height (obj): %g' % model2.objVal)
print('\tCheck Stem Height: ' + str(0.1*checkLeafSA + 0.01*checkLeafSA*savg))
print('\tCheck Flower Height: ' + str(checkFA + checkFSD*favg))
print('\tCheck Total Height (obj): ' + str(0.1*checkLeafSA + 0.01*checkLeafSA*savg + checkFA + checkFSD*favg))

Leaf standard normal sample average: -0.018163146219847298
Stem standard normal sample average: 0.012307744970036599
Flower standard normal sample average: -0.11583969464496291

Gurobi Result: Tulip Type: -0
Gurobi Result: Amount of Water/week (mL): 250
Gurobi Result: Outdoor?: -0
Gurobi Result: Total Leaf Surface Area Average: 147.5
	Check Result: 147.5
Gurobi Result: Total Leaf Surface Area Standard Deviation: 31.75
	Check Result: 31.75
Gurobi Result: Total Leaf Surface Area: 146.923
	Check Result: 146.92332010751986
Gurobi Result: Flower Petal Height Average: 3.125
	Check Result: 3.125
Gurobi Result: Flower Petal Height Standard Deviation: 0.75
	Check Result: 0.75

Gurobi Stem + Flower Height (obj): 17.7485
	Check Stem Height: 14.710414958292331
	Check Flower Height: 3.0381202290162777
	Check Total Height (obj): 17.74853518730861


In [211]:
from gurobipy import *
import numpy as np

'''
Finally, the gardener decides she wants to maximize the sum of the heights of the stem and the flower 
as well as the length of the roots.
Since she is now taking the roots into consideration, she must also decide how many pellets (p) of fertilizer
to put into the soil/month (2 <= p <= 5).
The addition of fertilizer affects the leaf surface area (lsa) distribution:
   lsa ~ N(100 + 0.04*x + 5*d - 10*d, 100 + 0.01*x + d) (if t = 1)
   lsa ~ N(145 + 0.01*x + 5*d - 10*d, 33 - 0.005*x + d) (if t = 0)
       - The addition of too much fertilizer causes the edges of the leaves to shrivel due to
         a phenomenon called "fertilizer burn", where the water sent from the roots to the 
         leaves is oversaturated with nutrients which causes the leaf tissues to dry
       - To ensure the plant does not completely dry up, we must add a constraint that we must add at
         least 150*p mL of water 
The root length (cm) is normally distributed:
    Root length ~ N(20 + p + 2*d , 1 + d)
        - Planting the tulip outside can increase the root length because of there is more space available
          and more nutrients at deeper altitudes. 
The stem and flower distributions remain the same. 
'''

# Create Model
model3 = Model("Model #3")
model3.setParam('OutputFlag', 0)

# Create Variables 
t = model3.addVar(vtype = GRB.BINARY, name = "Tulip Type")
x = model3.addVar(lb = 250, ub = 1000, name = "Amount of Water/week (mL)")
d = model3.addVar(vtype = GRB.BINARY, name = "Outdoor?")
p = model3.addVar(lb = 2, ub = 5, name = "Number of Fertilizer Pellets")
lsaavg = model3.addVar(name = "Total Leaf Surface Area Average")
lsastdev = model3.addVar(name = "Total Leaf Surface Area Standard Deviation")
lsa = model3.addVar(name = "Total Leaf Surface Area")
flavg = model3.addVar(name = "Flower Petal Height Average")
flstdev = model3.addVar(name = "Flower Petal Height Standard Deviation")

# Monte Carlo-ing Expectation
n = 100
i = 0
lavg = 0
savg = 0
favg = 0
ravg = 0
leaf = LinExpr()
stem = LinExpr()
flower = LinExpr()
roots = LinExpr()
while i < n:
    lnorm = np.random.standard_normal()
    snorm = np.random.standard_normal()
    fnorm = np.random.standard_normal()
    rnorm = np.random.standard_normal()
    
    leaf += lsaavg + lsastdev*lnorm
    stem += 0.1*(lsa) + 0.01*(lsa)*snorm
    flower += flavg + flstdev*fnorm 
    roots += 20 + 2*p + 2*d + (1 + d)*rnorm
    
    lavg += lnorm
    savg += snorm 
    favg += fnorm
    ravg += rnorm
    
    i += 1

lavg = (1/n)*lavg
savg = (1/n)*savg
favg = (1/n)*favg
ravg = (1/n)*ravg

print("Leaf standard normal sample average: " + str(lavg))
print("Stem standard normal sample average: " + str(savg))
print("Flower standard normal sample average: " + str(favg))
print("Roots standard normal sample average: " + str(ravg) + "\n")

# Set objective
model3.setObjective((1/n)*(stem + flower + roots), GRB.MAXIMIZE)

# Set constraints 
M = 10000

# Leaf Constraints 
# If t = 1, lsaavg = 100 + 0.06*x + 5*d - 10*p, else lsaavg = 145 + 0.01*x + 5*d - 10*p
model3.addConstr(lsaavg <= 100 + 0.04*x + 5*d - 10*p + M*(1 - t))
model3.addConstr(lsaavg >= 100 + 0.04*x + 5*d - 10*p - M*(1 - t))
model3.addConstr(lsaavg <= 145 + 0.01*x + 5*d - 10*p + M*t)
model3.addConstr(lsaavg >= 145 + 0.01*x + 5*d - 10*p - M*t)
# If t = 1, lsastdev = 100 + 0.01*x + d, else lsastdev = 33 - 0.005*x + d
model3.addConstr(lsastdev <=  100 + 0.01*x + d + M*(1 - t))
model3.addConstr(lsastdev >=  100 + 0.01*x + d - M*(1 - t))
model3.addConstr(lsastdev <= 33 - 0.005*x + d + M*t)
model3.addConstr(lsastdev >= 33 - 0.005*x + d - M*t)
# Setting lsa = leaf expression 
model3.addConstr(lsa <= (1/n)*leaf)
model3.addConstr(lsa >= (1/n)*leaf)

# Flower Constraints 
# If t = 1, flavg = 3 - 0.001*x + 1.5*(1 - d), else flavg = 3 - 0.0015*x + 0.5*(1 - d)
model3.addConstr(flavg <= 3 - 0.001*x + 1.5*(1 - d) + M*(1 - t))
model3.addConstr(flavg >= 3 - 0.001*x + 1.5*(1 - d) - M*(1 - t))
model3.addConstr(flavg <= 3 - 0.0015*x + 0.5*(1 - d) + M*t)
model3.addConstr(flavg >= 3 - 0.0015*x + 0.5*(1 - d) - M*t)
# If t = 1, flstdev = 1.35 + d, else flstdev = 0.75 + d
model3.addConstr(flstdev <= 1.35 + d + M*(1 - t))
model3.addConstr(flstdev >= 1.35 + d - M*(1 - t))
model3.addConstr(flstdev <= 0.75 + d + M*t)
model3.addConstr(flstdev >= 0.75 + d - M*t)

# Water vs. pellets constraint
model3.addConstr(x >= 150*p)

# Optimize model
model3.optimize()

# Print and check results
checkLA = 100*t.x + 0.04*x.x*t.x + 5*d.x*t.x - 10*p.x*t.x + 145*(1-t.x) + 0.01*x.x*(1-t.x) + 5*d.x*(1-t.x) - 10*p.x*(1-t.x)
checkLSD = 100*t.x + 0.01*x.x*t.x + d.x*t.x + 33*(1-t.x) - 0.005*x.x*(1-t.x) + d.x*(1-t.x)
checkFA = 3*t.x - 0.001*x.x*t.x + 1.5*(1-d.x)*t.x + 3*(1-t.x) - 0.0015*x.x*(1-t.x) + 0.5*(1-d.x)*(1 - t.x)
checkFSD = 1.35*t.x + d.x*t.x + 0.75*(1-t.x) + d.x*(1-t.x)
checkLeafSA = checkLA + checkLSD*lavg
checkRoots = 20 + 2*p.x + 2*d.x + (1 + d.x)*ravg
for v in model3.getVars():
    print('Gurobi Result: %s: %g' % (v.varName, v.x))
    if v.varName == "Total Leaf Surface Area Average":
        print('\tCheck Result: ' + str(checkLA))
    elif v.varName == 'Total Leaf Surface Area Standard Deviation':
        print('\tCheck Result: ' + str(checkLSD))
    elif v.varName == 'Total Leaf Surface Area':
        print('\tCheck Result: ' + str(checkLeafSA))
    elif v.varName == 'Flower Petal Height Average':
        print('\tCheck Result: ' + str(checkFA))
    elif v.varName == 'Flower Petal Height Standard Deviation':
        print('\tCheck Result: ' + str(checkFSD))


print('\nGurobi Stem + Flower + Roots Height (obj): %g' % model3.objVal)
print('\tCheck Stem Height: ' + str(0.1*checkLeafSA + 0.01*checkLeafSA*savg))
print('\tCheck Flower Height: ' + str(checkFA + checkFSD*favg))
print('\tCheck Roots Length: ' + str(checkRoots))
print('\tCheck Total Height (obj): ' + str(0.1*checkLeafSA + 0.01*checkLeafSA*savg + checkFA + checkFSD*favg + checkRoots))

Leaf standard normal sample average: 0.14744625658456983
Stem standard normal sample average: -0.04136890206622145
Flower standard normal sample average: -0.09550254661924536
Roots standard normal sample average: -0.0662851554551007

Gurobi Result: Tulip Type: 1
Gurobi Result: Amount of Water/week (mL): 1000
Gurobi Result: Outdoor?: 1
Gurobi Result: Number of Fertilizer Pellets: 5
Gurobi Result: Total Leaf Surface Area Average: 95
	Check Result: 95.0
Gurobi Result: Total Leaf Surface Area Standard Deviation: 111
	Check Result: 111.0
Gurobi Result: Total Leaf Surface Area: 111.367
	Check Result: 111.36653448088725
Gurobi Result: Flower Petal Height Average: 2
	Check Result: 2.0
Gurobi Result: Flower Petal Height Standard Deviation: 2.35
	Check Result: 2.35

Gurobi Stem + Flower + Roots Height (obj): 44.7336
	Check Stem Height: 11.090582335504783
	Check Flower Height: 1.7755690154447734
	Check Roots Length: 31.8674296890898
	Check Total Height (obj): 44.733581040039354


In [80]:
from gurobipy import *
import numpy as np

# Create Model
modelx = Model("Modelx")
#modelx.setParam('OutputFlag', 0)

# Create variables
x1 = modelx.addVar(lb = -1000, ub= 1000, name = "x1")
x2 = modelx.addVar(vtype = GRB.BINARY, name = "x2")
sca1 = modelx.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "y1 scale")
loc2 = modelx.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "y2 location")

# Monte carlo
n = 10 # Number of trials 
e1 = 0
e2 = 0
i = 0
y1 = LinExpr()
y2 = LinExpr()
while i < n:
    norm1 = np.random.standard_normal()
    norm2 = np.random.standard_normal()
    y1 += x1 + sca1*norm1
    y2 += loc2 + x1*norm2
    e1 += norm1 # Used to check the sample average for testing
    e2 += norm2
    i += 1
e1 = e1/n
e2 = e2/n
print("Sample average of e1 is: " + str(e1))
print("Sample average of e2 is: " + str(e2) + "\n")

#Set objective 
modelx.setObjective((1/n)*(y1 - y2), GRB.MAXIMIZE)

#Set constraints
M = 10000

#y1 constraints 
modelx.addConstr(sca1 <= -2*x1 + M*(1 - x2))
modelx.addConstr(sca1 >= -2*x1 - M*(1 - x2))
modelx.addConstr(sca1 <= 2*x1 + M*x2)
modelx.addConstr(sca1 >= 2*x1 - M*x2)

#y2 constraints
modelx.addConstr(loc2 <= 1.5*x1 + M*(1 - x2))
modelx.addConstr(loc2 >= 1.5*x1 - M*(1 - x2))
modelx.addConstr(loc2 <= 2*x1 + M*x2)
modelx.addConstr(loc2 >= 2*x1 - M*x2)

# Optimize model
modelx.optimize()

# Print Results 
for v in modelx.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % modelx.objVal)

Sample average of e1 is: 0.05557326916084613
Sample average of e2 is: -0.3345019890237368

Optimize a model with 8 rows, 4 columns and 24 nonzeros
Variable types: 3 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+03]
  RHS range        [1e+04, 1e+04]
Presolve time: 0.00s
Presolved: 8 rows, 4 columns, 24 nonzeros
Variable types: 3 continuous, 1 integer (1 binary)

Root relaxation: objective 1.902357e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1902.35702    0    1          - 1902.35702      -     -    0s
H    0     0                     554.3514727 1902.35702   243%     -    0s

Cutting planes:
  MIR: 1

Explored 0 nodes (3 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 1: 554.351 

In [74]:
from gurobipy import *
import numpy as np

# Create Model
modely = Model("Modely")
#model3.setParam('OutputFlag', 0)

# Create variables
loc = modely.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "Location")
d = modely.addVar(vtype = GRB.BINARY, name = "Difficult")
m = modely.addVar(vtype = GRB.BINARY, name = "Multiple Choice")
x = modely.addVar(lb = 60, ub = 120, name = "Time")

#Set coefficient e -> Monte carlo'd over n trials 
n = 10000 # Number of trials 
e1 = 0
e2 = 0
i = 0
y = LinExpr()
bonus = LinExpr()
while i < n:
    norm1 = np.random.standard_normal()
    norm2 = np.random.standard_normal()
    y += loc + (10*d + 5*m)*norm1
    bonus += d + norm2
    e1 += norm1 # Used to check the sample average for testing
    e2 += norm2
    i += 1
e1 = e1/n
e2 = e2/n
print("Sample average of e1 is: " + str(e1))
print("Sample average of e2 is: " + str(e2) + "\n")

#Set objective 
modely.setObjective((1/n)*y + (1/n)*bonus, GRB.MAXIMIZE)

#Set constraints
M = 10000

#y constraints 
modely.addConstr(loc <= 60 - 5*m + 0.1*x + M*(1 - d))
modely.addConstr(loc >= 60 - 5*m + 0.1*x - M*(1 - d))
modely.addConstr(loc <= 70 + 3*m + 0.001*x + M*d)
modely.addConstr(loc >= 70 + 3*m + 0.001*x - M*d)

# Optimize model
modely.optimize()

# Print Results 
for v in modely.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % modely.objVal)

Sample average of e1 is: 0.01885855880991378
Sample average of e2 is: 0.0060704045082175355

Optimize a model with 4 rows, 4 columns and 16 nonzeros
Variable types: 2 continuous, 2 integer (2 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+04]
  Objective range  [9e-02, 1e+00]
  Bounds range     [1e+00, 1e+02]
  RHS range        [7e+01, 1e+04]
Presolve time: 0.00s
Presolved: 4 rows, 4 columns, 16 nonzeros
Variable types: 2 continuous, 2 integer (2 binary)

Root relaxation: objective 7.398169e+01, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   73.98169    0    2          -   73.98169      -     -    0s
H    0     0                      68.2889488   73.98169  8.34%     -    0s
H    0     0                      73.1946560   73.98169  1.08%     -    0s
H    0     0                      73.2203632   73.98169  1.04%     -    0s

Expl

In [76]:
from gurobipy import *
import numpy as np

# Create Model
modelz = Model("Modelz")
#modelz.setParam('OutputFlag', 0)

# Create variables
x1 = modelz.addVar(lb = 1, ub = 10, vtype = GRB.INTEGER, name = "x1")
x2 = modelz.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "x2")
x3 = modelz.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "x3")
b3 = modelz.addVar(vtype = GRB.BINARY, name = "b3")
x4 = modelz.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "x4")
b4 = modelz.addVar(vtype = GRB.BINARY, name = "b4")
y2loc = modelz.addVar(lb = -1*GRB.INFINITY, ub = GRB.INFINITY, name = "y2loc")
by2 = modelz.addVar(vtype = GRB.BINARY, name = "by2")

#Set coefficient e -> Monte carlo'd over n trials 
n = 10000 # Number of trials 
x2e = 0
y1e = 0
y2e = 0
y3e = 0
i = 0
x2expr = LinExpr()
y1 = LinExpr()
y2 = LinExpr()
y3 = LinExpr()
bonus = LinExpr()
while i < n:
    normx = np.random.standard_normal()
    norm1 = np.random.standard_normal()
    norm2 = np.random.standard_normal()
    norm3 = np.random.standard_normal()
    
    x2expr += x1*normx
    y1 += (x3 - x4) + (x3 + x4)*norm1
    y2 += y2loc + (x4 - x3)*norm2
    y3 += x4*norm3
    
    x2e += normx
    y1e += norm1
    y2e += norm2
    y3e += norm3

    i += 1

print("Sample average of x2e is: " + str(x2e/n))
print("Sample average of y1e is: " + str(y1e/n))
print("Sample average of y2e is: " + str(y2e/n))
print("Sample average of y3e is: " + str(y3e/n) + "\n")

#Set objective 
modelz.setObjective((1/n)*(y1 - y2 + y3), GRB.MAXIMIZE)

#Set constraints
M = 10000

# x2 constraint
modelz.addConstr(x2 == (1/n)*x2expr)

# x3 constraints 
modelz.addConstr(x2 >= x1 - M*b3)
modelz.addConstr(x3 >= x2 - x1 - M*b3)
modelz.addConstr(x3 <= x2 - x1 + M*b3)
modelz.addConstr(x2 <= x1 + M*(1 - b3))
modelz.addConstr(x3 >= (1/2)*x1 - x2 - M*(1 - b3))
modelz.addConstr(x3 <= (1/2)*x1 - x2 + M*(1 - b3))

# x4 constraints 
modelz.addConstr(x2 <= -1*x1 + M*b4)
modelz.addConstr(x4 >= -1*x1 - x2 - M*b4)
modelz.addConstr(x4 <= -1*x1 - x2 + M*b4)
modelz.addConstr(x2 >= -1*x1 - M*(1 - b4))
modelz.addConstr(x4 >= 2*x2 - M*(1 - b4))
modelz.addConstr(x4 <= 2*x2 + M*(1 - b4))

# y2loc constraints 
modelz.addConstr(x3 >= x4 - M*by2)
modelz.addConstr(y2loc >= x4 - M*by2)
modelz.addConstr(y2loc <= x4 + M*by2)
modelz.addConstr(x3 <= x4 + M*(1 - by2))
modelz.addConstr(y2loc >= x3 - M*(1 - by2))
modelz.addConstr(y2loc <= x3 + M*(1 - by2))

# Optimize model
modelz.optimize()

# Print Results 
for v in modelz.getVars():
    print('%s: %g' % (v.varName, v.x))

print('Obj: %g' % modelz.objVal)

Sample average of x2e is: 0.009540473404689742
Sample average of y1e is: 0.0076242765638888715
Sample average of y2e is: 0.008624923623610609
Sample average of y3e is: -0.002560717290279621

Optimize a model with 19 rows, 8 columns and 62 nonzeros
Variable types: 4 continuous, 4 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+04, 1e+04]
Presolve removed 19 rows and 8 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 4.60199 
Pool objective bound 4.60199

Optimal solution found (tolerance 1.00e-04)
Best objective 4.601992538043e+00, best bound 4.601992538043e+00, gap 0.0000%
x1: 10
x2: 0.0954047
x3: 4.9046
b3: 1
x4: 0.190809
b4: 1
y2loc: 0.190809
by2: 0
Obj: 4.60199
